# Описание параметров

Если установить debug=True, сервер будет возвращать подробные сообщения об ошибках, что удобно для разработки, но в продакшене лучше оставить значение False, чтобы избежать утечки чувствительной информации

In [ ]:
from fastapi import FastAPI
app = FastAPI(debug=True)

Для улучшения документации API предусмотрен параметр title, который задаёт название API, отображаемое в Swagger UI и ReDoc.

In [ ]:
from fastapi import FastAPI
app = FastAPI(title="My First Project")

Параметр summary позволяет добавить краткое описание API, которое будет отображаться в документации. 

In [ ]:
from fastapi import FastAPI
app = FastAPI(summary="My CRUD application.")

Для более подробного описания используется параметр description, поддерживающий синтаксис Markdown (CommonMark).

In [ ]:
from fastapi import FastAPI
app = FastAPI(description="The CRUD application supports **writing**, *reading*, updating, and deleting posts.")


Версия вашего API задаётся через параметр version. Это версия вашего приложения, а не FastAPI или спецификации OpenAPI.

In [ ]:
from fastapi import FastAPI
app = FastAPI(version="0.0.1")

Параметр openapi_url определяет URL, по которому доступна схема OpenAPI в формате JSON. 

In [ ]:
from fastapi import FastAPI
app = FastAPI(openapi_url="/api/v1/openapi.json")

Для группировки операций в документации используется параметр openapi_tags. Он принимает список словарей, где каждый тег описывает группу маршрутов, например, [{"name": "users", "description": "User operations"}]. Это помогает организовать документацию, особенно если вы используете теги в маршрутах, таких как @app.get("/users/", tags=["users"]):

In [ ]:
from fastapi import FastAPI
app = FastAPI(openapi_tags=[{"name": "users", "description": "User operations"}])

Если API работает на нескольких серверах, например, для staging и production окружений, параметр serversпозволяет указать их адреса и описания. Например, можно задать два сервера: [{"url": "https://stag.example.com", "description": "Staging"}, {"url": "https://prod.example.com", "description": "Production"}]. Это удобно для тестирования API в разных окружениях

In [ ]:
from fastapi import FastAPI
app = FastAPI(servers=[
    {"url": "https://stag.example.com", "description": "Staging"},
    {"url": "https://prod.example.com", "description": "Production"}
])

Формат ответов API задаётся через параметр default_response_class. По умолчанию используется JSONResponse, но для более быстрой сериализации можно выбрать, например, ORJSONResponse

In [ ]:
from fastapi import FastAPI
from fastapi.responses import ORJSONResponse
app = FastAPI(default_response_class=ORJSONResponse)

Параметр redirect_slashes управляет автоматическим перенаправлением запросов с/без конечной косой черты. Если он включён (True по умолчанию), запрос к /items будет перенаправлен на /items/ с кодом состояния 307

In [ ]:
from fastapi import FastAPI
app = FastAPI(redirect_slashes=True)
@app.get("/items/")
async def read_items():
    return {"item_id": "Foo"}

Автоматическая документация доступна через Swagger UI и ReDoc, пути к которым задаются параметрами docs_url и redoc_url. По умолчанию они установлены как "/docs" и "/redoc", но их можно изменить или отключить, установив None

In [ ]:
from fastapi import FastAPI
app = FastAPI(docs_url="/custom-docs", redoc_url=None)

Для поддержки OAuth2 в Swagger UI используется параметр swagger_ui_oauth2_redirect_url, который задаёт путь для перенаправления при авторизации (по умолчанию "/docs/oauth2-redirect"). Можно указать свой путь:

In [ ]:
from fastapi import FastAPI
app = FastAPI(swagger_ui_oauth2_redirect_url="/custom-oauth-redirect")

Параметр swagger_ui_init_oauth позволяет настроить параметры OAuth2 для кнопки "Authorize" в Swagger UI, например, указав clientId

In [ ]:
from fastapi import FastAPI
app = FastAPI(swagger_ui_init_oauth={"clientId": "your-client-id"})

Промежуточное программное обеспечение (middleware) можно добавить через параметр middleware, хотя в FastAPI чаще используется метод app.add_middleware(). Например, можно добавить middleware для логирования:

In [ ]:
from fastapi import FastAPI
from starlette.middleware import Middleware
from starlette.middleware.base import BaseHTTPMiddleware
class CustomMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        response = await call_next(request)
        return response
app = FastAPI(middleware=[Middleware(CustomMiddleware)])

Обработка исключений задаётся через параметр exception_handlers, но обычно она настраивается через декоратор @app.exception_handler()

In [ ]:
from fastapi import FastAPI, HTTPException
app = FastAPI()
@app.exception_handler(HTTPException)
async def custom_exception_handler(request, exc):
    return {"detail": "Custom error"}

Контактная информация для API указывается через параметр contact, который принимает словарь с полями name, url и email. Это улучшает документацию, показывая, к кому обращаться:

In [ ]:
from fastapi import FastAPI
app = FastAPI(contact={
    "name": "John Doe",
    "url": "https://example.com",
    "email": "john.doe@example.com"
})

Информация о лицензии API задаётся через параметр license_info, где указываются название лицензии и её URL, например:

In [ ]:
from fastapi import FastAPI
app = FastAPI(license_info={
    "name": "MIT",
    "url": "https://opensource.org/licenses/MIT"
})

Условия использования API можно указать через параметр terms_of_service, задав URL условий, например:

In [ ]:
from fastapi import FastAPI
app = FastAPI(terms_of_service="https://example.com/terms/")

Если API работает за прокси, параметр root_path позволяет указать префикс пути, например, "/api/v1". Это влияет на документацию и клиентские запросы:

In [ ]:
from fastapi import FastAPI
app = FastAPI(root_path="/api/v1")

Параметр root_path_in_servers определяет, включать ли root_path в список серверов в документации OpenAPI. По умолчанию он включён (True):

In [ ]:
from fastapi import FastAPI
app = FastAPI(root_path="/api/v1", root_path_in_servers=False)

# Типы маршрутов в FastAPI

Теперь создадим файл crud.py с каркасом приложения, включающим маршруты для чтения, создания, обновления и удаления сообщений, следуя REST-практикам:

In [ ]:
from fastapi import (
    FastAPI,
    status,
    Body,
)

app = FastAPI()

# Это словарь, имитирующий базу данных
messages_db = {0: "First post in FastAPI"}

# Вернуть все сообщения
@app.get("/messages")
async def read_messages() -> dict:
    return messages_db

# Вернуть конкретное сообщение
@app.get("/messages/{message_id}")
async def read_message(message_id: int) -> str:
    if message_id not in messages_db:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Comment not found")
    return messages_db[message_id]

# Добавить сообщение (вариант, как не надо делать, так как все данные будут видны в url)
@app.post("/messages", status_code=status.HTTP_201_CREATED)  # для возврата кода 201 (Created), который указывает на успешное создание ресурса
async def create_message(message: str) -> str:
    current_index = max(messages_db) + 1 if messages_db else 0
    messages_db[current_index] = message
    return "Message created!"

# Добавить сообщение (вариант, корректный)
@app.post("/messages", status_code=status.HTTP_201_CREATED)  # для возврата кода 201 (Created), который указывает на успешное создание ресурса
async def create_message(message: str = Body(...)) -> str:  # обязательно используем класс Body для передачи данных через тело запроса
    current_index = max(messages_db) + 1 if messages_db else 0
    messages_db[current_index] = message
    return "Message created!"

# Обновить конкретное сообщение (полностью заменят текущее на переданое)
@app.put("/messages/{message_id}", status_code=status.HTTP_200_OK)  # для возврата кода 200 (OK)
async def update_message(message_id: int, message: str = Body(...)) -> str:  # обязательно используем класс Body для передачи данных через тело запроса
    if message_id not in messages_db:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Message not found")  # 404 если сообщение не найдено
    messages_db[message_id] = message
    return "Message updated!"

# Удалить одно сообщение
@app.delete("/messages/{message_id}", status_code=status.HTTP_200_OK)  # для возврата кода 200 (OK)
async def delete_message(message_id: int) -> str:
    if message_id not in messages_db:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Message not found")  # 404 если сообщение не найдено
    messages_db.pop(message_id)
    return f"Message ID={message_id} deleted!"

# Удалить все сообщения
@app.delete("/messages", status_code=status.HTTP_200_OK)
async def delete_messages() -> str:
    messages_db.clear()
    return "All messages deleted!"


# Pydantic

In [ ]:
from pydantic import BaseModel


class Message(BaseModel):
    id: int
    content: str
    tags: list[str] | None = None  # Необязательное поле, по умолчанию None
    priority: int = 0  # Поле с значением по умолчанию
    metadata: dict[str, int]  # Словарь с ключами-строками и значениями-числами

# Пример использования
message = Message(id=1, content="Hello, Pydantic!")
print(message)  # Вывод: id=1 content='Hello, Pydantic!'

Давайте рассмотрим 2 самых важных метода для работы с Pydantic моделями, это  model_dump() и model_dump_json().

Начнем с model_dump(), этот метод преобразует объект Pydantic в Python-словарь (dict). Он полезен, когда нужно получить данные модели в виде словаря для дальнейшей обработки в коде. Используем этот метод для нашей модели:

In [ ]:
print(message.model_dump())  # Сериализация в словарь: {'id': 1, 'content': 'Hello, Pydantic!'}

Синтаксис:
model_dump(*, include=None, exclude=None, by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False) -> dict

                  
Параметры:
* include: Указывает, какие поля включить в результирующий словарь (множество, список или словарь с именами полей).
* exclude: Указывает, какие поля исключить из результирующего словаря.
* by_alias: Если True, ключи словаря будут использовать псевдонимы (алиасы), заданные в модели (например, если в модели есть alias для поля).
* exclude_unset: Если True, исключаются поля, которые не были явно установлены (например, если значение осталось значением по умолчанию).
* exclude_defaults: Если True, исключаются поля, значения которых совпадают со значениями по умолчанию, определёнными в модели.
* exclude_none: Если True, исключаются поля со значением None.


А теперь рассмотрим метод model_dump_json(), этот метод сериализует объект Pydantic в JSON-строку. Он полезен, когда нужно получить данные в формате JSON для отправки по сети или сохранения. Используем этот метод для нашей модели:

In [ ]:
print(message.model_dump_json(indent=2))  # Сериализация в JSON: {"id": 1, "content": "Hello, Pydantic!"}

Имеет те же параметры что и model_dump, только добавлен один дополнительный indent для красивого отображения

### Специализированные типы Pydantic
Pydantic предоставляет встроенные типы для сложных случаев, которые автоматически проверяют данные:

* EmailStr: Проверяет, что строка — валидный email (требуется pip install email-validator).
* HttpUrl: Проверяет, что строка — валидный URL.
* PositiveInt: Проверяет, что целое число положительное.
* NonNegativeInt: Проверяет, что целое число не отрицательное.
* NegativeFloat: Проверяет, что число с плавающей точкой отрицательное.
* constr: Ограничивает строки (например, по длине или регулярному выражению).
* conint: Ограничивает целые числа (например, по диапазону).

In [ ]:
from pydantic import BaseModel, EmailStr, HttpUrl, PositiveInt


class User(BaseModel):
    email: EmailStr
    website: HttpUrl
    age: PositiveInt


### Вложенные модели

In [ ]:
from pydantic import BaseModel


class Author(BaseModel):
    name: str
    email: EmailStr


class Message(BaseModel):
    id: int
    content: str
    author: Author  # Вложенная модель


message = Message(
    id=1,
    content="Hello",
    author=Author(name="Alice", email="alice@example.com")
)

# Валидация Полей Pydantic

In [ ]:
from pydantic import BaseModel


class Message(BaseModel):
    id: int
    content: str


# Успешная валидация
message = Message(id=1, content="Hello")
print(message)  # Вывод: id=1 content='Hello'

# Автоматическое преобразование
message = Message(id="123", content="Hello")  # Строку "123" Pydantic преобразует в int
print(message)  # Вывод: id=123 content='Hello'

# Ошибка валидации
message = Message(id="not_a_number", content="Hello")


In [ ]:
from pydantic import BaseModel, SecretStr


class UserWithPassword(BaseModel):
    username: str
    password: SecretStr


user = UserWithPassword(
    username="alice",
    password="super_secret_password"
)

print(user)  
# username='alice' password=SecretStr('**********')

# Чтобы получить реальное значение пароля:
real_password = user.password.get_secret_value()
print(real_password)  # super_secret_password

## Валидация через Field

Модуль pydantic.Field позволяет задавать ограничения для полей, такие как длина строки, диапазон чисел, регулярные выражения и т.д. По умолчанию все поля в модели обязательные.

### Основные параметры Field
default: Значение по умолчанию для поля, если оно не указано.
* Пример: Field(default="some_value")
* Делает поле необязательным.

default_factory: Функция, вызываемая для создания значения по умолчанию (для динамических значений, например, UUID).
* Пример: Field(default_factory=lambda: uuid4())
* Делает поле необязательным.

title: Заголовок поля для документации (например, OpenAPI).
* Пример: Field(..., title="User ID")

description: Описание поля для документации.
* Пример: Field(..., description="Unique identifier for a user")

const: Указывает, что поле является константой и всегда имеет значение из default.
* Пример: Field(default="constant_value", const=True)

gt: Ограничение "больше" для числовых полей.
* Пример: Field(..., gt=0)

ge: Ограничение "больше или равно" для числовых полей.
* Пример: Field(..., ge=0)

lt: Ограничение "меньше" для числовых полей.
* Пример: Field(..., lt=100)

le: Ограничение "меньше или равно" для числовых полей.
* Пример: Field(..., le=100)

multiple_of: Числовое значение должно быть кратно указанному числу.
* Пример: Field(..., multiple_of=5)

min_length: Минимальная длина строки.
* Пример: Field(..., min_length=3)

max_length: Максимальная длина строки.
* Пример: Field(..., max_length=50)

pattern: Регулярное выражение для валидации строк.
* Пример: Field(..., pattern=r"^[a-zA-Z0-9]+$")
* Проверяет, что строка соответствует заданному регулярному выражению.

strict: Включает строгий режим валидации (требует точное соответствие типов).
* Пример: Field(..., strict=True)

exclude: Исключает поле из сериализации (например, в model.dict()).
* Пример: Field(..., exclude=True)

include: Указывает, включать ли поле в сериализацию (используется редко, обычно с exclude).
* Пример: Field(..., include=True)

json_schema_extra: Дополнительные параметры для JSON-схемы (например, для OpenAPI).
* Пример: Field(..., json_schema_extra={"example": "example_value"})

deprecated: Помечает поле как устаревшее для документации.
* Пример: Field(..., deprecated=True)

frozen: Запрещает изменение значения поля после создания экземпляра.
* Пример: Field(..., frozen=True)

In [ ]:
from pydantic import BaseModel, Field


class Message(BaseModel):
    id: int = Field(..., gt=0)  # Явно указываем что это обязательное поле, число больше 0
    content: str = Field(min_length=1, max_length=500, pattern=r"^[a-zA-Z0-9\s!,.?]*$")  # Строка 1-500 символов, только буквы, цифры, пробелы и знаки
    priority: float = Field(default=0.0, ge=0.0, le=10.0)  # Число от 0 до 10


In [ ]:
from pydantic import BaseModel, Field


class Article(BaseModel):
    text: str
    slug: str = Field(pattern=r'^[-a-zA-Z0-9_]+$')


# Пример использования:
valid_article = Article(text="Some text", slug="valid-slug_123")
invalid_article = Article(text="Some text", slug="Invalid Slug!") # Ошибка
invalid_article2 = Article(text="Some text", slug=" ") # Ошибка

### Кастомная валидация с @field_validator
Для сложных проверок, выходящих за рамки Field, используется декоратор @field_validator. Он позволяет писать пользовательские функции для проверки отдельных полей.

In [ ]:
from pydantic import BaseModel, Field, field_validator


class Message(BaseModel):
    content: str = Field(min_length=1, max_length=500)

    @field_validator("content")
    @classmethod
    def check_forbidden_words(cls, value):
        forbidden_words = ["spam", "offensive"]
        if any(word in value.lower() for word in forbidden_words):
            raise ValueError("Message contains forbidden words")
        return value

In [ ]:
message = Message(content="Hello, world!")  # OK
message = Message(content="This is spam")  # Ошибка: Message contains forbidden words

### Проверка нескольких полей

In [13]:
from pydantic import BaseModel, field_validator


class Message(BaseModel):
    id: int
    content: str

    @field_validator("content")
    @classmethod
    def check_content_id_match(cls, value, info):
        if str(info.data.get("id", "")) not in value:
            raise ValueError("Content must contain the ID")
        return value


In [ ]:
message = Message(id=1, content="Message 1")  # OK
message = Message(id=1, content="No ID here")  # Ошибка: Content must contain the ID

### Валидация на уровне модели через @model_validator
В Pydantic декоратор @model_validator используется для валидации на уровне модели, когда нужно проверить взаимосвязь между несколькими полями. Он позволяет выполнять проверки, учитывающие значения нескольких полей одновременно, до или после стандартной валидации.

In [14]:
from pydantic import BaseModel, model_validator


class User(BaseModel):
    name: str
    age: int
    email: str

    @model_validator(mode='after')
    def check_age_and_email(self):
        if self.age < 18 and self.email:
            raise ValueError("Несовершеннолетним нельзя указывать email")
        return self


# Пример использования
data = {"name": "Alice", "age": 16, "email": "alice@example.com"}
try:
    user = User.model_validate(data)
except ValueError as e:
    print(e)  # Выведет: Несовершеннолетним нельзя указывать email

1 validation error for User
  Value error, Несовершеннолетним нельзя указывать email [type=value_error, input_value={'name': 'Alice', 'age': ...l': 'alice@example.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


### Дополнительные возможности валидации
Кастомные типы с Annotated

Pydantic активно использует typing.Annotated для создания кастомных типов с помощью StringConstraints, NumberConstraints и т.д.:



In [ ]:
from pydantic import BaseModel
from typing import Annotated
from pydantic.types import StringConstraints


Username = Annotated[str, StringConstraints(min_length=3, max_length=20, pattern=r"^[a-zA-Z0-9_]+$")]


class User(BaseModel):
    username: Username


user = User(username="john_doe")  # OK
user = User(username="ab")  # Ошибка: слишком короткое имя


In [ ]:
from pydantic import BaseModel, Field
from decimal import Decimal


class Product(BaseModel):
    price: Decimal = Field(max_digits=6, decimal_places=2)  # Например, 1234.56


product = Product(price=1234.56)  # OK
product = Product(price=1234567.89)  # Ошибка: слишком много цифр


In [ ]:
from pydantic import BaseModel, Field


class Message(BaseModel):
    id: int = Field(strict=True)


message = Message(id=123)  # OK
message = Message(id="123")  # Ошибка: String is not allowed in strict mode
